In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input as preprocess_hybrid
from tensorflow.keras.preprocessing import image

Load the models

In [2]:
#Load Resnet50
resnet_model = tf.keras.models.load_model("../Feature Extraction models/resnet50_model")
#Remove Classifiation Layers
resnet50 = tf.keras.models.Sequential(resnet_model.layers[:-5])
resnet50.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [2]:
#Load Inception-V3
inceptionv3_model = tf.keras.models.load_model("../Feature Extraction models/inceptionv3_model")
#Remove Classifiation Layers
inceptionv3 = tf.keras.models.Sequential(inceptionv3_model.layers[:-5])
inceptionv3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________


In [3]:
#Load Inception-Resnet-V2
hybrid_model = tf.keras.models.load_model("../Feature Extraction models/hybrid_model")
#Remove Classifiation Layers
hybrid = tf.keras.models.Sequential(hybrid_model.layers[:-5])
hybrid.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 54,336,736
Trainable params: 54,276,192
Non-trainable params: 60,544
_________________________________________________________________


In [4]:
def preprocess_img(img_path, preprocessing_function):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocessing_function(x)
    return x

def extract_features_from_img(img, model):
    # Use the the transfer learning model to extract features
    features = model.predict(img)

    # Flatten the feature vector
    features = features.flatten()

    return features

def extract_features(path_to_cat, model, preprocessing_function, batch=0, batch_size=0, save_path='model'): #Use batch option to extract parts of the dataset
    data_array = ([][])
    if(batch>0):
        x=batch_size #Number of images to use during feature extraction
        start=batch-1
        start=start*x #Start posistion of the dataset
        start=int(start/2)
        stop=int(start+x/2)
    else:
        x=0
        start=0
        stop=0
    for cat in os.listdir(path_to_cat):
        # Defining path to images in category
        path_to_images = os.path.join(path_to_cat, cat)
        
        # Defining labels
        if cat == 'MSIMUT':
            label = 1
        else:
            label = 0
            
        z=-1
        # Loop through images in category
        for i in os.listdir(path_to_images):
            if(batch>0):
                z=z+1
            if(z<start and start!=0 and batch>0):
                continue
            if(z==stop and batch>0):
                break
                
            # Path to image
            image_path = os.path.join(path_to_images, i)
            
            # Reading and preprocessing image
            x = preprocess_img(image_path, preprocessing_function)
            
            # Extracting features
            features = extract_features_from_img(x, model)
            
            # Store features and label in our lists
            data_array[0].append(features)
            data_array[1].append(label)
    if(batch>0):
        np.save(save_path + '_features_' + str(batch) + ".npy", data_array[0])
        np.save(save_path + '_labels_' + str(batch) + ".npy", data_array[1])
    else:
        np.save(save_path + '_features' + ".npy", data_array[0])
        np.save(save_path + '_labels' + ".npy", data_array[1])

Extract Features from Training Dataset

In [ ]:
path_to_train = "../Dataset/training"
path_to_test = "../Dataset/evaluation"
seed = 666 #Use seed to get the results that can be recreated

rs_path = "../Features/resnet50"

inc_path = "../Features/inceptionv3"

hybrid_path = "../Features/hybrid"

rs_test_path = "../Features/resnet50"

inc_test_path = "../Features/inceptionv3_test"

hybrid_test_path = "../Features/hybrid_test"

In [ ]:
#Extract training features using Resnet50
extract_features(path_to_train, resnet50, preprocess_resnet, save_path=rs_path)

In [ ]:
#Extract training features using Inception
extract_features(path_to_train, inceptionv3, preprocess_inception, save_path=inc_path)

In [ ]:
#Extract training features using Hybrid
extract_features(path_to_train, hybrid, preprocess_hybrid, save_path=hybrid_path)

Extract features from evaluation dataset

In [ ]:
#Extract test features using Resnet50
extract_features(path_to_test, resnet50, preprocess_resnet, save_path=rs_test_path)

In [ ]:
#Extract test features using Inception
extract_features(path_to_test, inceptionv3, preprocess_inception, save_path=inc_test_path)

In [ ]:
#Extract test features using Hybrid
extract_features(path_to_test, hybrid, preprocess_hybrid, save_path=hybrid_test_path)

Reshape features and save them to single files. Use only if batch parameter was used!

In [ ]:
#Use the functions below only if you used batch parameter and you want to combine the feature files into one file

In [3]:
def save_to_one_file(train_path, lbl_path, name, num_of_files, num_of_features, num_of_samples):
    train_files = [train_path + str(i+1) +".npy" for i in range(num_of_files)]
    train_lbl = [lbl_path + str(i+1) +".npy" for i in range(num_of_files)]
    train_X = np.array([np.load(fname) for fname in train_files])
    train_X = train_X.reshape(num_of_samples, num_of_features)
    train_Y = np.array([np.load(fname) for fname in train_lbl])
    train_Y = train_Y.reshape(num_of_samples)
    np.save("../Features/" + name + "_features.npy", train_X)
    np.save("../Features/" + name + "_labels.npy", train_Y)

In [5]:
#Reshape features and save them to one file
resnet_features_path = "../Features/training/resnet_features_"
resnet_lbl_path = "../Features/training/resnet_labels_"

inception_features_path = "../Features/training/inceptionv3_features_"
inception_lbl_path = "../Features/training/inceptionv3_labels_"

hybrid_features_path = "../Features/training/hybrid_features_"
hybrid_lbl_path = "../Features/training/hybrid_labels_"

resnet_test_features_path = "../Features/evaluation/resnet50_test_features_"
resnet_test_lbl_path = "../Features/evaluation/resnet50_test_labels_"

inception_test_features_path = "../Features/evaluation/inceptionv3_test_features_"
inception_test_lbl_path = "../Features/evaluation/inceptionv3_test_labels_"

hybrid_test_features_path = "../Features/evaluation/hybrid_test_features_"
hybrid_test_lbl_path = "../Features/evaluation/hybrid_test_labels_"

samples = 60000
files = 10
tst_files = 2
tst_samples=12000

In [ ]:
#Save training features to one file
save_to_one_file(resnet_features_path, resnet_lbl_path, "resnet50", files, 2048, samples)

In [ ]:
save_to_one_file(inception_features_path, inception_lbl_path, "inceptionv3",files,2048, samples)

In [ ]:
save_to_one_file(hybrid_features_path, hybrid_lbl_path, "hybrid",files,1536, samples)

In [6]:
#Save test features to one file
save_to_one_file(resnet_test_features_path, resnet_test_lbl_path, "resnet50_test", tst_files, 2048, tst_samples)

In [ ]:
save_to_one_file(inception_test_features_path, inception_test_lbl_path, "inceptionv3_test", tst_files, 2048, tst_samples)

In [ ]:
save_to_one_file(hybrid_test_features_path, hybrid_test_lbl_path, "hybrid_test", tst_files, 1536, tst_samples)